### 1. 텍스트 파일에서의 토큰 추출

In [1]:
# simple_preprocess: tokenize data through Handling punctuations and lowercasing the text 
from gensim.utils import simple_preprocess 

In [2]:
sample_text = "These guyes are smart students of AI department in Jeonju university."

In [3]:
# Remove accent marks from tokens using deaccent() function
print(simple_preprocess(sample_text, deacc = True))

['these', 'guyes', 'are', 'smart', 'students', 'of', 'ai', 'department', 'in', 'jeonju', 'university']


### 2. 한국어 Word2Vec 만들기
* 영화 리뷰 데이터 활용
* 참고: https://monetd.github.io/python/nlp/Word-Embedding-Word2Vec-%EC%8B%A4%EC%8A%B5/

In [9]:
# 필요한 라이브러리 불러오기
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt    # konlpy 설치 필요(pip install konlpy)

In [12]:
# 데이터 가져오기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
train_data = pd.read_table('ratings.txt')
train_data[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


각각의 리뷰에 대한 id값과 실제 리뷰 내용 그리고 긍정/부정 여부를 나타내는 label 값으로 이루어짐

In [13]:
# 전처리
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
# 정규 표현식을 통한 한글 외 문자 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

In [14]:
# 토큰화 및 불용어 제거 (시간 소요)
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

okt = Okt()
tokenized_data = []
for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    tokenized_data.append(temp_X)

In [15]:
tokenized_data[:5] # 토큰화된 상위 5개 데이터를 확인

[['어리다', '때', '보고', '지금', '다시', '보다', '재밌다', 'ㅋㅋ'],
 ['디자인',
  '을',
  '배우다',
  '학생',
  ',',
  '외국',
  '디자이너',
  '그',
  '일군',
  '전통',
  '을',
  '통해',
  '발전',
  '문화',
  '산업',
  '부럽다',
  '.',
  '사실',
  '우리나라',
  '에서도',
  '그',
  '어렵다',
  '시절',
  '끝',
  '까지',
  '열정',
  '을',
  '지키다',
  '노라노',
  '같다',
  '전통',
  '있다',
  '저',
  '같다',
  '사람',
  '꿈',
  '을',
  '꾸다',
  '이루다',
  '나가다',
  '수',
  '있다',
  '것',
  '감사하다',
  '.'],
 ['폴리스스토리', '시리즈', '1', '부터', '뉴', '까지', '버리다', '하나', '없다', '..', '최고', '.'],
 ['오다',
  '..',
  '연기',
  '진짜',
  '개',
  '쩔다',
  '..',
  '지루하다',
  '생각',
  '몰입',
  '보다',
  '..',
  '그렇다',
  '이렇다',
  '진짜',
  '영화',
  '지'],
 ['안개', '자욱하다', '밤하늘', '뜨다', '있다', '초승달', '같다', '영화', '.']]

In [18]:
# Gensim을 이용하여 Word2Vec 훈련
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, window = 5, min_count = 5, workers = 4, sg = 0)
print(model.wv.vectors.shape)  # 완성된 임베딩 매트릭스의 크기 확인

(17615, 100)


In [19]:
# 완성된 모델을 이용해 “한석규”과 비슷한 단어들을 출력
print(model.wv.most_similar("한석규"))

[('설경구', 0.8955587148666382), ('엄정화', 0.8701022863388062), ('송강호', 0.8641658425331116), ('차승원', 0.8603668808937073), ('김명민', 0.8602263331413269), ('정재영', 0.8549153208732605), ('신들리다', 0.8544403910636902), ('박상민', 0.8537116050720215), ('이요원', 0.8499498963356018), ('다우니', 0.847318172454834)]
